In [1]:
!pip install transformers datasets torch transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import get_scheduler
import torch
from torch.optim import Adam  # Use Adam optimizer

In [3]:
test_data = pd.read_csv("banking_conversation_5000nonhate+2232hate.csv")

In [4]:
# prompt: rename "text" column to "cleaned_text" in test_data

test_data = test_data.rename(columns={"text": "cleaned_text"})
test_data

,cleaned_text,label
0,Are you seriously this morons? No wonder your ...,1
1,(exasperated) Fine. The name of the bank where...,0
2,"If you don’t reverse the charge, I will take l...",1
3,I don’t want to speak to immigrants. Get me a ...,1
4,"That's great, thank you! Actually, I'm looking...",0
...,...,...
7227,"You too, Tonia. Bye.",0
7228,You customer service people are just morons. I...,1
7229,"Yes, I'm here. Sorry about that. my call got d...",0
7230,Are you seriously this morons? No wonder your ...,1


In [5]:

test_data = test_data.dropna(subset=['cleaned_text', 'label'])
test_data['cleaned_text'] = test_data['cleaned_text'].astype(str)

test_texts = test_data["cleaned_text"].tolist()
test_labels = test_data["label"].tolist()

In [6]:


# Select 50,000 rows randomly
#data = data.sample(50000, random_state=42)

# Ensure the dataset contains 'text' and 'label' columns (replace these names if different in your dataset)
data = test_data[['cleaned_text', 'label']]
data['label'] = data['label'].astype(int)  # Convert labels to integers

# Remove rows with missing values if any
data = data.dropna()

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['cleaned_text'], data['label'], test_size=0.01, random_state=42)

# Initialize the XLNet tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Ensure all texts are strings and drop any NaN or null values
train_texts = train_texts.dropna().astype(str)
#val_texts = val_texts.dropna().astype(str)

# Tokenize the text data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
#val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

# Define a custom dataset class
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Create DataLoaders for training and validation
train_dataset = TextDataset(train_encodings, list(train_labels))
#val_dataset = TextDataset(val_encodings, list(val_labels))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8, drop_last=True)
#val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8, drop_last=True)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [7]:
second_data = pd.read_csv("banking77_5000nonhate+2231hate.csv")
second_data = second_data.rename(columns={"text": "cleaned_text"})
second_data = second_data.dropna(subset=['cleaned_text', 'label'])
second_data['cleaned_text'] = second_data['cleaned_text'].astype(str)

second_texts = second_data["cleaned_text"].tolist()
second_labels = second_data["label"].tolist()

In [8]:


# Select 50,000 rows randomly
#data = data.sample(50000, random_state=42)

# Ensure the dataset contains 'text' and 'label' columns (replace these names if different in your dataset)
second = second_data[['cleaned_text', 'label']]
second['label'] = second['label'].astype(int)  # Convert labels to integers

# Remove rows with missing values if any
second = second.dropna()

# Split the data into training and validation sets
train_texts2, val_texts2, train_labels2, val_labels2 = train_test_split(
    second['cleaned_text'], second['label'], test_size=0.99, random_state=42)

# Initialize the XLNet tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Ensure all texts are strings and drop any NaN or null values
#train_texts2 = train_texts.dropna().astype(str)
val_texts2 = val_texts2.dropna().astype(str)

# Tokenize the text data
#train_encodings2 = tokenizer(list(train_texts2), truncation=True, padding=True, max_length=128)
val_encodings2 = tokenizer(list(val_texts2), truncation=True, padding=True, max_length=128)

# Define a custom dataset class
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Create DataLoaders for training and validation
#train_dataset2 = TextDataset(train_encodings2, list(train_labels2))
val_dataset2 = TextDataset(val_encodings2, list(val_labels2))

#train_loader2 = DataLoader(train_dataset2, batch_size=32, shuffle=True, num_workers=8, drop_last=True)
val_loader = DataLoader(val_dataset2, batch_size=32, shuffle=False, num_workers=8, drop_last=True)



/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [9]:


# Load the pre-trained XLNet model for binary classification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)

# Use GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define the optimizer and learning rate scheduler
optimizer = Adam(model.parameters(), lr=5e-5)  # Using Adam optimizer
num_training_steps = len(train_loader) * 3  # Assuming 3 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Disable torch dynamo by skipping its usage
torch._dynamo.disable()


pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

model.eval()  # Set model to evaluation mode
all_val_preds = []
all_val_labels = []
val_loss = 0

with torch.no_grad():  # Disable gradient calculation for validation
    for batch in tqdm(val_loader, desc="Initial Validation"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        val_loss += loss.item()

        preds = torch.argmax(logits, dim=-1).detach().cpu().numpy()
        labels = batch['labels'].detach().cpu().numpy()
        all_val_preds.extend(preds)
        all_val_labels.extend(labels)

# Calculate validation metrics
avg_val_loss = val_loss / len(val_loader)
val_acc = accuracy_score(all_val_labels, all_val_preds)
val_precision = precision_score(all_val_labels, all_val_preds, average='binary')
val_recall = recall_score(all_val_labels, all_val_preds, average='binary')
val_f1 = f1_score(all_val_labels, all_val_preds, average='binary')

print(f"Initial Validation | Loss: {avg_val_loss:.4f} | Accuracy: {val_acc:.4f} | Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}")


Initial Validation: 100%|██████████| 223/223 [00:52<00:00,  4.22it/s]

Initial Validation | Loss: 0.7287 | Accuracy: 0.4020 | Precision: 0.2570 | Recall: 0.4984 | F1: 0.3392


In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# Initialize lists to store metrics
train_loss_list = []
train_metrics = []

epochs = 3  # You can change this based on your preference

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    all_train_preds = []
    all_train_labels = []

    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        train_loss += loss.item()

        preds = torch.argmax(logits, dim=-1).detach().cpu().numpy()
        labels = batch['labels'].detach().cpu().numpy()
        all_train_preds.extend(preds)
        all_train_labels.extend(labels)

    avg_train_loss = train_loss / len(train_loader)
    train_acc = accuracy_score(all_train_labels, all_train_preds)
    train_precision = precision_score(all_train_labels, all_train_preds, average='binary')
    train_recall = recall_score(all_train_labels, all_train_preds, average='binary')
    train_f1 = f1_score(all_train_labels, all_train_preds, average='binary')

    print(f"Epoch {epoch + 1} | Train Loss: {avg_train_loss:.4f} | Accuracy: {train_acc:.4f} | Precision: {train_precision:.4f} | Recall: {train_recall:.4f} | F1: {train_f1:.4f}")

    train_loss_list.append(avg_train_loss)
    train_metrics.append({'accuracy': train_acc, 'precision': train_precision, 'recall': train_recall, 'f1': train_f1})

    model.eval()  # Set model to evaluation mode
    all_val_preds = []
    all_val_labels = []
    val_loss = 0
    with torch.no_grad():  # Disable gradient calculation for validation
        for batch in tqdm(val_loader, desc="Initial Validation"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()

            preds = torch.argmax(logits, dim=-1).detach().cpu().numpy()
            labels = batch['labels'].detach().cpu().numpy()
            all_val_preds.extend(preds)
            all_val_labels.extend(labels)

    # Calculate validation metrics
    avg_val_loss = val_loss / len(val_loader)
    val_acc = accuracy_score(all_val_labels, all_val_preds)
    val_precision = precision_score(all_val_labels, all_val_preds, average='binary')
    val_recall = recall_score(all_val_labels, all_val_preds, average='binary')
    val_f1 = f1_score(all_val_labels, all_val_preds, average='binary')

    print(f"Initial Validation | Loss: {avg_val_loss:.4f} | Accuracy: {val_acc:.4f} | Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}")


Training Epoch 1:   0%|          | 0/223 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Training Epoch 1: 100%|██████████| 223/223 [02:36<00:00,  1.42it/s]


Epoch 1 | Train Loss: 0.0318 | Accuracy: 0.9871 | Precision: 0.9911 | Recall: 0.9668 | F1: 0.9788


Initial Validation:   0%|          | 0/223 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Initial Validation: 100%|██████████| 223/223 [00:55<00:00,  4.03it/s]


Initial Validation | Loss: 0.0732 | Accuracy: 0.9846 | Precision: 0.9523 | Recall: 1.0000 | F1: 0.9756


Training Epoch 2:   0%|          | 0/223 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Training Epoch 2: 100%|██████████| 223/223 [02:40<00:00,  1.39it/s]


...

Initial Validation | Loss: 0.0886 | Accuracy: 0.9825 | Precision: 0.9462 | Recall: 1.0000 | F1: 0.9723


Training Epoch 3:   0%|          | 0/223 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Training Epoch 3: 100%|██████████| 223/223 [02:39<00:00,  1.40it/s]


Epoch 3 | Train Loss: 0.0025 | Accuracy: 0.9994 | Precision: 0.9991 | Recall: 0.9991 | F1: 0.9991


Initial Validation:   0%|          | 0/223 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Initial Validation: 100%|██████████| 223/223 [00:55<00:00,  4.03it/s]

Initial Validation | Loss: 0.0277 | Accuracy: 0.9947 | Precision: 0.9830 | Recall: 1.0000 | F1: 0.9914


In [12]:
second_data = pd.read_csv("detoxified_texts_v2_unique.csv")
second_data = second_data.rename(columns={"text": "cleaned_text"})
second_data = second_data.dropna(subset=['cleaned_text', 'label'])
second_data['cleaned_text'] = second_data['cleaned_text'].astype(str)

second_texts = second_data["cleaned_text"].tolist()
second_labels = second_data["label"].tolist()

In [14]:


# Select 50,000 rows randomly
#data = data.sample(50000, random_state=42)

# Ensure the dataset contains 'text' and 'label' columns (replace these names if different in your dataset)
second = second_data[['cleaned_text', 'label']]
second['label'] = second['label'].astype(int)  # Convert labels to integers

# Remove rows with missing values if any
second = second.dropna()

# Split the data into training and validation sets
train_texts2, val_texts2, train_labels2, val_labels2 = train_test_split(
    second['cleaned_text'], second['label'], test_size=0.98, random_state=42)

# Initialize the XLNet tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Ensure all texts are strings and drop any NaN or null values
#train_texts2 = train_texts.dropna().astype(str)
val_texts2 = val_texts2.dropna().astype(str)

# Tokenize the text data
#train_encodings2 = tokenizer(list(train_texts2), truncation=True, padding=True, max_length=128)
val_encodings2 = tokenizer(list(val_texts2), truncation=True, padding=True, max_length=128)

# Define a custom dataset class
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Create DataLoaders for training and validation
#train_dataset2 = TextDataset(train_encodings2, list(train_labels2))
val_dataset2 = TextDataset(val_encodings2, list(val_labels2))

#train_loader2 = DataLoader(train_dataset2, batch_size=32, shuffle=True, num_workers=8, drop_last=True)
val_loader = DataLoader(val_dataset2, batch_size=32, shuffle=False, num_workers=8, drop_last=True)



<ipython-input-14-d4db82fe04c7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second['label'] = second['label'].astype(int)  # Convert labels to integers
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [15]:
model.eval()  # Set model to evaluation mode
all_val_preds = []
all_val_labels = []
val_loss = 0

with torch.no_grad():  # Disable gradient calculation for validation
    for batch in tqdm(val_loader, desc="Initial Validation"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        val_loss += loss.item()

        preds = torch.argmax(logits, dim=-1).detach().cpu().numpy()
        labels = batch['labels'].detach().cpu().numpy()
        all_val_preds.extend(preds)
        all_val_labels.extend(labels)

# Calculate validation metrics
avg_val_loss = val_loss / len(val_loader)
val_acc = accuracy_score(all_val_labels, all_val_preds)
val_precision = precision_score(all_val_labels, all_val_preds, average='binary')
val_recall = recall_score(all_val_labels, all_val_preds, average='binary')
val_f1 = f1_score(all_val_labels, all_val_preds, average='binary')

print(f"Initial Validation | Loss: {avg_val_loss:.4f} | Accuracy: {val_acc:.4f} | Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}")

Initial Validation:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Initial Validation: 100%|██████████| 2/2 [00:01<00:00,  1.15it/s]

Initial Validation | Loss: 0.0001 | Accuracy: 1.0000 | Precision: 1.0000 | Recall: 1.0000 | F1: 1.0000


In [19]:
second_data = pd.read_csv("detoxified_texts_v2_unique.csv")
second_data = second_data.rename(columns={"detoxified_text": "cleaned_text"})
second_data = second_data.dropna(subset=['cleaned_text', 'label'])
second_data['cleaned_text'] = second_data['cleaned_text'].astype(str)

second_texts = second_data["cleaned_text"].tolist()
second_labels = second_data["label"].tolist()

In [20]:


# Select 50,000 rows randomly
#data = data.sample(50000, random_state=42)

# Ensure the dataset contains 'text' and 'label' columns (replace these names if different in your dataset)
second = second_data[['cleaned_text', 'label']]
#second['label'] = second['label'].astype(int)  # Convert labels to integers
second['label'] = 0

# Remove rows with missing values if any
second = second.dropna()

# Split the data into training and validation sets
train_texts2, val_texts2, train_labels2, val_labels2 = train_test_split(
    second['cleaned_text'], second['label'], test_size=0.98, random_state=42)

# Initialize the XLNet tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Ensure all texts are strings and drop any NaN or null values
#train_texts2 = train_texts.dropna().astype(str)
val_texts2 = val_texts2.dropna().astype(str)

# Tokenize the text data
#train_encodings2 = tokenizer(list(train_texts2), truncation=True, padding=True, max_length=128)
val_encodings2 = tokenizer(list(val_texts2), truncation=True, padding=True, max_length=128)

# Define a custom dataset class
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Create DataLoaders for training and validation
#train_dataset2 = TextDataset(train_encodings2, list(train_labels2))
val_dataset2 = TextDataset(val_encodings2, list(val_labels2))

#train_loader2 = DataLoader(train_dataset2, batch_size=32, shuffle=True, num_workers=8, drop_last=True)
val_loader = DataLoader(val_dataset2, batch_size=32, shuffle=False, num_workers=8, drop_last=True)



<ipython-input-20-4a33ff6f2a92>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second['label'] = 0
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [21]:
model.eval()  # Set model to evaluation mode
all_val_preds = []
all_val_labels = []
val_loss = 0

with torch.no_grad():  # Disable gradient calculation for validation
    for batch in tqdm(val_loader, desc="Initial Validation"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        val_loss += loss.item()

        preds = torch.argmax(logits, dim=-1).detach().cpu().numpy()
        labels = batch['labels'].detach().cpu().numpy()
        all_val_preds.extend(preds)
        all_val_labels.extend(labels)

# Calculate validation metrics
avg_val_loss = val_loss / len(val_loader)
val_acc = accuracy_score(all_val_labels, all_val_preds)
val_precision = precision_score(all_val_labels, all_val_preds, average='binary')
val_recall = recall_score(all_val_labels, all_val_preds, average='binary')
val_f1 = f1_score(all_val_labels, all_val_preds, average='binary')

print(f"Initial Validation | Loss: {avg_val_loss:.4f} | Accuracy: {val_acc:.4f} | Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}")

Initial Validation: 100%|██████████| 2/2 [00:01<00:00,  1.87it/s]

Initial Validation | Loss: 0.0615 | Accuracy: 0.9844 | Precision: 0.0000 | Recall: 0.0000 | F1: 0.0000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
